# **Evaluación Comprehensiva - Metodología Completa**

Sistema de evaluación robusto con:
- Benchmarks simples y complejos
- Múltiples heurísticas de comparación
- Análisis estadístico riguroso
- Visualizaciones comparativas

In [ ]:
%run librerias.ipynb

In [ ]:
%run benchmark_loader.ipynb

In [ ]:
%run gnn_model.ipynb

In [ ]:
%run heuristics.ipynb

## **1. PREPARACIÓN DE DATOS PARA GNN**

In [ ]:
def preparar_datos_gnn(G):
    """
    Convierte un grafo NetworkX a formato PyTorch Geometric.
    """
    X = extraer_features(G)
    X = normalizar_features(X)
    
    edge_index = np.array(list(G.edges())).T
    
    if edge_index.size == 0:
        edge_index = np.array([[], []], dtype=np.int64)
    
    edge_index_reverse = np.array([edge_index[1], edge_index[0]])
    edge_index_bidirectional = np.concatenate([edge_index, edge_index_reverse], axis=1)
    
    x_tensor = torch.FloatTensor(X)
    edge_index_tensor = torch.LongTensor(edge_index_bidirectional)
    
    from torch_geometric.data import Data
    data = Data(x=x_tensor, edge_index=edge_index_tensor)
    
    return data

## **2. FUNCIÓN DE COLORACIÓN GREEDY PARA GNN**

In [ ]:
def greedy_coloring_gnn(edge_index, num_nodes, ordering):
    """
    Algoritmo greedy de coloración dado un ordenamiento.
    Compatible con edge_index de PyTorch Geometric.
    """
    if isinstance(edge_index, torch.Tensor):
        edge_index = edge_index.cpu().numpy()
    
    adj = {i: set() for i in range(num_nodes)}
    for i in range(edge_index.shape[1]):
        u, v = edge_index[0, i], edge_index[1, i]
        if u != v:
            adj[u].add(v)
            adj[v].add(u)
    
    coloring = {}
    for node in ordering:
        neighbor_colors = {coloring[v] for v in adj[node] if v in coloring}
        
        color = 0
        while color in neighbor_colors:
            color += 1
        coloring[node] = color
    
    return max(coloring.values()) + 1 if coloring else 0

## **3. EVALUADOR MULTI-HEURÍSTICA**

In [ ]:
def evaluar_todas_heuristicas(G, model=None, repeticiones=5):
    """
    Evalúa múltiples heurísticas en un grafo.
    
    Heurísticas evaluadas:
    1. Random (baseline)
    2. Greedy Natural Order
    3. Largest Degree First
    4. Welsh-Powell
    5. DSATUR
    6. GNN-guided (si se proporciona modelo)
    """
    resultados = []
    
    for _ in range(repeticiones):
        order_random = ordenamiento_aleatorio(G)
        t0 = time.time()
        coloring_random = greedy_coloring(G, order_random)
        t1 = time.time()
        metrics_random = evaluar_coloracion(G, coloring_random)
        resultados.append({
            'metodo': 'Random',
            'colores': metrics_random['num_colores'],
            'tiempo': t1 - t0,
            'valido': metrics_random['valido']
        })
    
    order_natural = list(G.nodes())
    t0 = time.time()
    coloring_natural = greedy_coloring(G, order_natural)
    t1 = time.time()
    metrics_natural = evaluar_coloracion(G, coloring_natural)
    resultados.append({
        'metodo': 'Greedy Natural',
        'colores': metrics_natural['num_colores'],
        'tiempo': t1 - t0,
        'valido': metrics_natural['valido']
    })
    
    order_degree = ordenamiento_grado_desc(G)
    t0 = time.time()
    coloring_degree = greedy_coloring(G, order_degree)
    t1 = time.time()
    metrics_degree = evaluar_coloracion(G, coloring_degree)
    resultados.append({
        'metodo': 'Largest Degree First',
        'colores': metrics_degree['num_colores'],
        'tiempo': t1 - t0,
        'valido': metrics_degree['valido']
    })
    
    order_wp = ordenamiento_welsh_powell(G)
    t0 = time.time()
    coloring_wp = greedy_coloring(G, order_wp)
    t1 = time.time()
    metrics_wp = evaluar_coloracion(G, coloring_wp)
    resultados.append({
        'metodo': 'Welsh-Powell',
        'colores': metrics_wp['num_colores'],
        'tiempo': t1 - t0,
        'valido': metrics_wp['valido']
    })
    
    t0 = time.time()
    coloring_dsatur = dsatur_coloring(G)
    t1 = time.time()
    metrics_dsatur = evaluar_coloracion(G, coloring_dsatur)
    resultados.append({
        'metodo': 'DSATUR',
        'colores': metrics_dsatur['num_colores'],
        'tiempo': t1 - t0,
        'valido': metrics_dsatur['valido']
    })
    
    if model is not None:
        try:
            data = preparar_datos_gnn(G)
            model.eval()
            
            with torch.no_grad():
                scores = model(data.x, data.edge_index)
                ordering = torch.argsort(scores, descending=True).tolist()
            
            t0 = time.time()
            colores_gnn = greedy_coloring_gnn(data.edge_index, data.num_nodes, ordering)
            t1 = time.time()
            
            resultados.append({
                'metodo': 'GNN-guided',
                'colores': colores_gnn,
                'tiempo': t1 - t0,
                'valido': True
            })
        except Exception as e:
            print(f"Error en GNN: {e}")
    
    return resultados

## **4. EVALUACIÓN EN SUITE DE BENCHMARKS**

In [ ]:
def evaluar_suite_completa(benchmarks, model=None, repeticiones=5):
    """
    Evalúa todas las heurísticas en una suite de benchmarks.
    """
    resultados_completos = []
    
    for i, (G, nombre) in enumerate(benchmarks):
        print(f"\n[{i+1}/{len(benchmarks)}] Evaluando: {nombre}")
        print(f"  Nodos: {G.number_of_nodes()}, Aristas: {G.number_of_edges()}")
        
        stats = estadisticas_grafo(G, nombre)
        resultados_heuristicas = evaluar_todas_heuristicas(G, model, repeticiones)
        
        for res in resultados_heuristicas:
            resultado_completo = {**stats, **res}
            resultados_completos.append(resultado_completo)
    
    return pd.DataFrame(resultados_completos)

## **5. ANÁLISIS ESTADÍSTICO**

In [ ]:
def analisis_estadistico(df):
    """
    Genera análisis estadístico de los resultados.
    """
    print("\n" + "="*80)
    print("ANÁLISIS ESTADÍSTICO POR MÉTODO")
    print("="*80)
    
    resumen = df.groupby('metodo').agg({
        'colores': ['mean', 'std', 'min', 'max'],
        'tiempo': ['mean', 'std'],
        'valido': 'sum'
    }).round(4)
    
    print(resumen)
    
    print("\n" + "="*80)
    print("RANKING DE MÉTODOS (por colores promedio)")
    print("="*80)
    
    ranking = df.groupby('metodo')['colores'].mean().sort_values()
    for i, (metodo, colores) in enumerate(ranking.items(), 1):
        print(f"{i}. {metodo:20s} - {colores:.2f} colores promedio")
    
    return resumen, ranking

## **6. COMPARACIÓN DETALLADA POR GRAFO**

In [ ]:
def comparacion_por_grafo(df):
    """
    Muestra comparación detallada por cada grafo.
    """
    print("\n" + "="*80)
    print("COMPARACIÓN DETALLADA POR GRAFO")
    print("="*80)
    
    grafos_unicos = df['nombre'].unique()
    
    for grafo in grafos_unicos:
        df_grafo = df[df['nombre'] == grafo]
        
        print(f"\n{'─'*80}")
        print(f"Grafo: {grafo}")
        
        info = df_grafo.iloc[0]
        print(f"  Nodos: {info['nodos']}, Aristas: {info['aristas']}, "
              f"Densidad: {info['densidad']:.4f}, Grado máx: {info['grado_max']}")
        
        if info['chi_teorico'] is not None:
            print(f"  χ(G) teórico: {info['chi_teorico']}")
        print(f"  Cota superior: Δ+1 = {info['cota_superior']}")
        
        print(f"\n  Resultados por método:")
        
        resumen_metodos = df_grafo.groupby('metodo').agg({
            'colores': ['mean', 'min'],
            'tiempo': 'mean'
        }).round(4)
        
        for metodo in resumen_metodos.index:
            colores_mean = resumen_metodos.loc[metodo, ('colores', 'mean')]
            colores_min = resumen_metodos.loc[metodo, ('colores', 'min')]
            tiempo = resumen_metodos.loc[metodo, ('tiempo', 'mean')]
            print(f"    {metodo:20s}: {colores_mean:6.2f} colores (min: {colores_min:.0f}) - {tiempo:.6f}s")
        
        mejor_metodo = resumen_metodos[('colores', 'mean')].idxmin()
        mejor_resultado = resumen_metodos.loc[mejor_metodo, ('colores', 'mean')]
        print(f"\n  ✓ Mejor método: {mejor_metodo} con {mejor_resultado:.2f} colores")

## **7. VISUALIZACIÓN DE RESULTADOS**

In [ ]:
def visualizar_resultados(df):
    """
    Genera visualizaciones de los resultados.
    """
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    resumen_metodos = df.groupby('metodo')['colores'].agg(['mean', 'std'])
    resumen_metodos = resumen_metodos.sort_values('mean')
    
    axes[0, 0].barh(resumen_metodos.index, resumen_metodos['mean'], 
                     xerr=resumen_metodos['std'], capsize=5)
    axes[0, 0].set_xlabel('Número de Colores (promedio)')
    axes[0, 0].set_title('Comparación de Métodos - Colores Usados')
    axes[0, 0].grid(axis='x', alpha=0.3)
    
    tiempo_metodos = df.groupby('metodo')['tiempo'].mean().sort_values()
    axes[0, 1].barh(tiempo_metodos.index, tiempo_metodos.values)
    axes[0, 1].set_xlabel('Tiempo (segundos)')
    axes[0, 1].set_title('Comparación de Métodos - Tiempo de Ejecución')
    axes[0, 1].grid(axis='x', alpha=0.3)
    
    for metodo in df['metodo'].unique():
        df_metodo = df[df['metodo'] == metodo]
        axes[1, 0].scatter(df_metodo['nodos'], df_metodo['colores'], 
                          label=metodo, alpha=0.6)
    axes[1, 0].set_xlabel('Número de Nodos')
    axes[1, 0].set_ylabel('Colores Usados')
    axes[1, 0].set_title('Escalabilidad - Nodos vs Colores')
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3)
    
    for metodo in df['metodo'].unique():
        df_metodo = df[df['metodo'] == metodo]
        axes[1, 1].scatter(df_metodo['densidad'], df_metodo['colores'], 
                          label=metodo, alpha=0.6)
    axes[1, 1].set_xlabel('Densidad del Grafo')
    axes[1, 1].set_ylabel('Colores Usados')
    axes[1, 1].set_title('Densidad vs Colores')
    axes[1, 1].legend()
    axes[1, 1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('resultados_evaluacion.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✓ Gráficos guardados en 'resultados_evaluacion.png'")

## **8. EJECUCIÓN PRINCIPAL - EVALUACIÓN SIMPLE**

In [ ]:
print("\n" + "#"*80)
print("# EVALUACIÓN COMPREHENSIVA - NIVEL SIMPLE")
print("#"*80)

benchmarks_simple = cargar_benchmark('suite', nivel='simple')
print(f"\nBenchmarks cargados: {len(benchmarks_simple)}")

df_resultados_simple = evaluar_suite_completa(benchmarks_simple, model=None, repeticiones=3)

resumen, ranking = analisis_estadistico(df_resultados_simple)
comparacion_por_grafo(df_resultados_simple)

try:
    visualizar_resultados(df_resultados_simple)
except Exception as e:
    print(f"\nNo se pudieron generar gráficos: {e}")

df_resultados_simple.to_csv('resultados_simple.csv', index=False)
print("\n✓ Resultados guardados en 'resultados_simple.csv'")

## **9. EJECUCIÓN OPCIONAL - EVALUACIÓN MEDIO**

In [ ]:
print("\n" + "#"*80)
print("# EVALUACIÓN COMPREHENSIVA - NIVEL MEDIO")
print("#"*80)

benchmarks_medio = cargar_benchmark('suite', nivel='medio')
print(f"\nBenchmarks cargados: {len(benchmarks_medio)}")

df_resultados_medio = evaluar_suite_completa(benchmarks_medio, model=None, repeticiones=3)

resumen_medio, ranking_medio = analisis_estadistico(df_resultados_medio)
comparacion_por_grafo(df_resultados_medio)

df_resultados_medio.to_csv('resultados_medio.csv', index=False)
print("\n✓ Resultados guardados en 'resultados_medio.csv'")

## **10. EJECUCIÓN OPCIONAL - EVALUACIÓN COMPLEJO**

In [ ]:
print("\n" + "#"*80)
print("# EVALUACIÓN COMPREHENSIVA - NIVEL COMPLEJO")
print("#"*80)

benchmarks_complejo = cargar_benchmark('suite', nivel='complejo')
print(f"\nBenchmarks cargados: {len(benchmarks_complejo)}")

df_resultados_complejo = evaluar_suite_completa(benchmarks_complejo, model=None, repeticiones=3)

resumen_complejo, ranking_complejo = analisis_estadistico(df_resultados_complejo)
comparacion_por_grafo(df_resultados_complejo)

df_resultados_complejo.to_csv('resultados_complejo.csv', index=False)
print("\n✓ Resultados guardados en 'resultados_complejo.csv'")

## **11. RESUMEN FINAL Y CONCLUSIONES**

In [ ]:
print("\n" + "#"*80)
print("# RESUMEN FINAL DE LA EVALUACIÓN")
print("#"*80)

print("\n📊 METODOLOGÍA IMPLEMENTADA:")
print("  ✓ Benchmarks simples: grafos clásicos y pequeños")
print("  ✓ Benchmarks medios: redes sintéticas complejas (100 nodos)")
print("  ✓ Benchmarks complejos: grafos grandes y desafiantes (500 nodos)")
print("  ✓ Soporte para DIMACS y SNAP datasets")

print("\n🔬 HEURÍSTICAS EVALUADAS:")
print("  1. Random (baseline)")
print("  2. Greedy Natural Order")
print("  3. Largest Degree First")
print("  4. Welsh-Powell")
print("  5. DSATUR")
print("  6. GNN-guided (cuando hay modelo entrenado)")

print("\n📈 MÉTRICAS ANALIZADAS:")
print("  • Número de colores (promedio, std, min, max)")
print("  • Tiempo de ejecución")
print("  • Validez de la coloración")
print("  • Escalabilidad con tamaño del grafo")
print("  • Comportamiento según densidad")

print("\n💾 ARCHIVOS GENERADOS:")
print("  • resultados_simple.csv")
print("  • resultados_medio.csv (opcional)")
print("  • resultados_complejo.csv (opcional)")
print("  • resultados_evaluacion.png")

print("\n" + "#"*80)